In [ ]:
import os
import psycopg2
import random

# Step 1: Function to create database connection
def create_connection():
    try:
        connection = psycopg2.connect(
            dbname="",
            user="",
            password="",
            host="",
            port=""
        )
        return connection
    except Exception as e:
        print("Error while connecting to the database:", e)
        return None

# Step 2: Function to create the property_images table
def create_image_table():
    connection = create_connection()
    if connection is None:
        return
    cursor = connection.cursor()
    try:
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS property_images (
            image_id SERIAL PRIMARY KEY,
            property_id INT NOT NULL,
            image_data BYTEA NOT NULL,
            FOREIGN KEY (property_id) REFERENCES property(property_id) ON DELETE CASCADE
        );
        """)
        connection.commit()
        print("property_images table created successfully.")
    except Exception as e:
        print("Error creating property_images table:", e)
    finally:
        cursor.close()
        connection.close()

# Step 3: Function to fetch image data from a local folder
def fetch_images_from_folder(folder_path):
    images = []
    try:
        for filename in os.listdir(folder_path):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                file_path = os.path.join(folder_path, filename)
                with open(file_path, 'rb') as image_file:
                    image_data = image_file.read()
                images.append(image_data)
        return images
    except Exception as e:
        print("Error reading images from folder:", e)
        return []

# Step 4: Function to fetch all property IDs from the database
def fetch_all_property_ids():
    connection = create_connection()
    if connection is None:
        return []
    cursor = connection.cursor()
    try:
        cursor.execute("SELECT property_id FROM property;")
        properties = cursor.fetchall()
        return [row[0] for row in properties]  # Extract property IDs
    except Exception as e:
        print("Error fetching property IDs:", e)
        return []
    finally:
        cursor.close()
        connection.close()

# Step 5: Function to assign images to properties in the database
def assign_images_to_properties(images, property_ids):
    connection = create_connection()
    if connection is None:
        return
    cursor = connection.cursor()
    try:
        for property_id in property_ids:
            assigned_images = random.sample(images, min(3, len(images)))  # Assign up to 3 images
            for image_data in assigned_images:
                cursor.execute("""
                    INSERT INTO property_images (property_id, image_data)
                    VALUES (%s, %s);
                """, (property_id, psycopg2.Binary(image_data)))
            print(f"Assigned {len(assigned_images)} images to property ID {property_id}.")
        connection.commit()
    except Exception as e:
        print("Error assigning images to properties:", e)
    finally:
        cursor.close()
        connection.close()

# Step 6: Main function to execute the process
def main():
    create_image_table()  # Ensure the property_images table exists
    folder_path = 'C:/Users/sa/Downloads/archive (4)/socal2/socal_pics'  # Adjust folder path
    images = fetch_images_from_folder(folder_path)
    
    if not images:
        print("No images found in the folder.")
        return

    property_ids = fetch_all_property_ids()
    
    if not property_ids:
        print("No properties found in the database.")
        return

    assign_images_to_properties(images, property_ids)

if __name__ == "__main__":
    main()


property_images table created successfully.
Assigned 3 images to property ID 919906.
Assigned 3 images to property ID 1232114.
Assigned 3 images to property ID 2207925.
Assigned 3 images to property ID 2207927.
Assigned 3 images to property ID 6319714.
Assigned 3 images to property ID 7456157.
Assigned 3 images to property ID 7456158.
Assigned 3 images to property ID 7893141.
Assigned 3 images to property ID 7960925.
Assigned 3 images to property ID 8154390.
Assigned 3 images to property ID 8178604.
Assigned 3 images to property ID 8256152.
Assigned 3 images to property ID 8259224.
Assigned 3 images to property ID 8270899.
Assigned 3 images to property ID 8496999.
Assigned 3 images to property ID 8502021.
Assigned 3 images to property ID 8785637.
Assigned 3 images to property ID 9046015.
Assigned 3 images to property ID 9050193.
Assigned 3 images to property ID 9064022.
Assigned 3 images to property ID 9093343.
Assigned 3 images to property ID 9151480.
Assigned 3 images to property ID 

In [6]:
! pip install Pillow

DEPRECATION: Loading egg at c:\users\sa\appdata\local\programs\python\python312\lib\site-packages\yapf-0.40.2-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [ ]:
import psycopg2
from PIL import Image
from io import BytesIO

# Function to create a database connection
def create_connection():
    try:
        connection = psycopg2.connect(
            dbname="",         # Database name
            user="",           # Database user
            password="",  # Database password
            host="",      # Host (IP address or localhost)
            port=""                # Port (usually 5432 for PostgreSQL)
        )
        return connection
    except Exception as e:
        print("Error while connecting to the database:", e)
        return None

# Function to fetch and display a specific row from the property_images table
def fetch_and_display_image(property_id):
    connection = create_connection()
    if connection is None:
        return

    cursor = connection.cursor()
    try:
        # Query to fetch the image data from property_images where property_id matches
        cursor.execute("""
            SELECT image_data FROM property_images WHERE property_id = %s LIMIT 1;
        """, (property_id,))
        
        # Fetch the row (first result)
        row = cursor.fetchone()

        # If a row is found, display the image
        if row:
            image_data = row[0]  # image_data is in the first column
            if image_data:
                # Convert the binary image data to an Image object using PIL
                image = Image.open(BytesIO(image_data))
                image.show()  # This will open the image in the default viewer
            else:
                print("No image data found for this property.")
        else:
            print(f"No image found for property ID {property_id}.")
    
    except Exception as e:
        print("Error fetching or displaying image:", e)
    
    finally:
        cursor.close()
        connection.close()

# Main function to run the script
def main():
    # Enter the property_id you want to search for
    property_id = 9525910 
    fetch_and_display_image(property_id)

if __name__ == "__main__":
    main()
